# Bilderkennung (Kontur)

## Einzelne Karten extrahieren und als Dateien speichern
In dem gegebenen Bild werden Karten erkannt, und nur diejenigen, die eine bestimmte Mindestgröße haben, werden als separate Dateien gespeichert.

#### Threshold
Der Schwellenwert (Threshold) ist ein "Kriterium", das je nach Situation variieren kann.  

Die Funktion "threshold(img, Schwellenwert, Wert oberhalb des Schwellenwerts) gibt "ret" und "binary"(Binärbild) zurück. Für Rückgabewerte, die nicht verwendet werden, kann man mit _ verarbeiten

Farbwerte nach RGB :
- Schwarz : 0
- Dunkelgrau : 127
- Hellgrau : 195
- Weiß : 255

Bei "cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)" ist der übergebene Schwellenwert der entscheidende Wert:   
- Pixel über dem Schwellenwert werden weiß  
- Pixel unter dem Schwellenwert werden schwarz
> Beispiel : Wenn der Schieberegler 194 anzeigt, werden alle Pixel mit einem Wert von 195 oder höher (also hellgrau) dann weiß

#### Otsu Algorithm
- Es kann mühsam sein, den richtigen Schwellenwert manuell über einen Schieberegler zu finden.
-> Der Otsu-Algorithmus findet den optimalen Schwellenwert automatisch

#### Konturenfindungsmodi
1. cv2.RETR_EXTERNAL : Findet nur die äußersten Konturen
1. cv2.RETR_LIST : Findet alle Konturen ohne Hierarchieinformationen
    - Keine Hierarchieinformationen : Gibt nur die Namen der Konturen zurück, ohne deren Beziehungen
1. cv2.RETR_TREE : Findet alle Konturen mit Hierarchieinformationen in Baumstruktur
    - Mit Hierarchieinformationen : Gibt die Beziehungen zwischen den Konturen zurück wie ein Stammbaum

#### Approximationsmethoden für Konturenfindung
1. cv2.CHAIN_APPROX_NONE : Gibt alle Koordinaten der Kontur zurück
1. cv2.CHAIN_APPROX_SIMPLE : Gibt nur die Koordinaten der Eckpunkte der Kontur zurück ohne Redundanz

Rechtecke :
- Bei Verwendung von NONE für Rechtecke werden alle Koordinaten der Kontur zurückgegeben
- Bei perfekten Rechtecken, die aus geraden Linien bestehen, reichen die vier Eckpunkte aus, um das Rechteck zu beschreiben. Daher wird in diesem Fall SIMPLE verwendet, um Speicherplatz zu sparen

#### Begrenzungsrechteck
Das Begrenzungsrechteck umschließt die Kontur
> boundingRect() : Gibt die Informationen des Rechtecks, das die Kontur umschließt, zurück (x, y, width, height : (x, y) sind die Koordinaten der oberen linken Ecke des Rechtecks)

#### Fläche
> contourArea() : Wenn man nur Konturen mit einer bestimmten Mindestgröße in einem Bild finden möchte

In [3]:
import cv2

img = cv2.imread('card.png')
target_img = img.copy() # kopiertes Bild

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, otsu = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Konturenerkennung : Die Funktion "findContours()" gibt Informationen über die Konturen ("contours") und deren hierarhische Struktur ("hierarchy") zurück
contours, hierarchy = cv2.findContours(otsu, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

# Konturen zeichnen
COLOR = (0, 200, 0)

idx = 1
# Flächen finden
for contour in contours:
    if cv2.contourArea(contour) > 25000: # Kartengröße : width 130 x height 205 = 26650
            x, y, width, height = cv2.boundingRect(contour)
            cv2.rectangle(target_img, (x, y), (x + width, y + height), COLOR, 2) # Rechteck zeichnen
            
            # Bildausschnitt ausschneiden
            crop = img[y : y + height, x : x + width]
            # height und width des Bildausschnitts ("img") defnieren
            cv2.imshow(f'card_cropped_{idx}', crop)
            # Datei speichern
            cv2.imwrite(f'card_cropped_{idx}.png', crop)
            idx += 1

cv2.imshow('original card', img)
cv2.imshow('contoured card', target_img)

cv2.waitKey(0)
cv2.destroyAllWindows()